In [ ]:
import cv2
import os
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib inline
from IPython.display import HTML, Video, display
from google.colab.patches import cv2_imshow

In [ ]:
%%sh
# Download the data - you need to do this only once
mkdir 03
wget --no-verbose --output-document=03/image.jpg https://github.com/chrirupp/cv_course/raw/main/data/image.jpg
wget --no-verbose --output-document=03/image_windows.jpg https://github.com/chrirupp/cv_course/raw/main/data/image_windows.jpg
wget --no-verbose --output-document=03/image_bridge.jpg https://github.com/chrirupp/cv_course/raw/main/data/image_bridge.jpg
wget --no-verbose --output-document=03/image_branch.jpg https://github.com/chrirupp/cv_course/raw/main/data/image_branch.jpg
wget --no-verbose --output-document=03/image_lanterns.jpg https://github.com/chrirupp/cv_course/raw/main/data/image_lanterns.jpg

In [ ]:
class Visualizer():
    def __init__(self, num_rows=1, num_cols=1, figsize=(5,5), title='', tight=False, cm=None):
        fig, self.axs = plt.subplots(num_rows, num_cols, figsize=figsize, squeeze = False)
        # remove ticks
        plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
        # set colormap
        if cm is not None:
            plt.set_cmap(cm)
        # set supertitle
        fig.suptitle(title)
        if tight:
            fig.subplots_adjust(top=0.88)

    def add_image_subplot(self, i, j, image, normalize=False, title_str=''):
        if normalize:
            image = self.normalize_image(image)
        self.axs[i, j].imshow(image)
        self.axs[i, j].set_title(title_str)

    def add_stem_subplot(self, i, j, x, y, title_str=''):
      self.axs[i, j].stem(x, y)
      self.axs[i, j].set_title(title_str)

    @staticmethod
    def normalize_image(image):
        img = np.float64(image) - np.min(image)
        img /= np.max(img)
        return img

def show_video(filename, width=640):
    tmp_mp4 = f"{filename[:-4]}_H264{filename[-4:]}"
    os.system(f"ffmpeg -i {filename} -vcodec libx264 {tmp_mp4}")
    os.remove(filename)
    os.rename(tmp_mp4, filename)
    display(Video(filename, embed=True, width=width))


In [ ]:
# make checkerboard
checkerboard = np.round(np.sqrt(np.arange(512, 0, -1, dtype=np.float32)*2)).astype(np.uint8)
checkerboard = checkerboard[:, np.newaxis] + checkerboard[np.newaxis, :]
checkerboard = (checkerboard % 2) * 255
checkerboard = np.stack([checkerboard, checkerboard, checkerboard], axis=2)

vis = Visualizer(1, 8, figsize=(32, 4), tight=True)
vis.add_image_subplot(0, 0, checkerboard, title_str='Original Image')

# downsample
for i in range(1, 8):
    vis.add_image_subplot(0, i, cv2.resize(checkerboard[::2**i, ::2**i, :], (512, 512), interpolation=cv2.INTER_NEAREST), title_str=f'Downsampled by {2**i}')

In [ ]:
# 1D discrete Fourier transform

num_samples = 16
values = (np.arange(num_samples, dtype=np.float32) / num_samples) * 2 * np.pi
function = np.sin(values)
vis = Visualizer(1, 2, figsize=(10, 5))
vis.add_stem_subplot(0, 0, values, function, title_str='sin(x)')

function_fft = fft.fft(function)
frequency = fft.fftfreq(values.shape[0], 1/num_samples)
amplitude = np.abs(function_fft)
plt.stem(frequency, amplitude)
vis.add_stem_subplot(0, 1, frequency, amplitude, title_str='DFT Amplitude sin(x)')


# 1D discrete Fourier with pixels

discrete_function_pixels = np.round(np.abs(function*0.5+0.5)*255).astype(np.uint8)
vis = Visualizer(1, 3, figsize=(15, 5), cm='gray')
vis.add_image_subplot(0, 0, cv2.resize(discrete_function_pixels[np.newaxis, :], (16*num_samples, 16), interpolation=cv2.INTER_NEAREST), title_str='Space Domain')

dft_matrix = np.exp(-2j * np.pi * np.outer(values, values))
vis.add_image_subplot(0, 1, np.real(dft_matrix), normalize=True, title_str='DFT (Real)')
vis.add_image_subplot(0, 2, np.imag(dft_matrix), normalize=True, title_str='DFT (Imaginary)')


In [ ]:
# 2D basis functions

num_samples = 256
values = (np.arange(num_samples, dtype=np.float32) / num_samples)
vis = Visualizer(3, 8, figsize=(15, 5), title='Basis functions', cm='gray', tight=True)

for u in range(0, 3):
    for v in range(0, 8):
        basis_function = np.exp(2j * np.pi * (u * values[:, np.newaxis] + v * values[np.newaxis, :]))
        vis.add_image_subplot(u, v, np.real(basis_function), normalize=True)


In [ ]:
# FFT examples on images

def fft_image(image):
    dft = fft.fft2(np.float32(image))
    dft_shift = np.fft.fftshift(dft)
    magnitude = np.abs(dft_shift)
    phase = np.angle(dft_shift)
    return magnitude, phase

def show_fft(image_filename):
    image = cv2.imread(image_filename, cv2.IMREAD_GRAYSCALE)
    mag, pha = fft_image(image)
    vis = Visualizer(1, 3, figsize=(12, 4), cm='gray')
    vis.add_image_subplot(0, 0, image, title_str='Image')
    vis.add_image_subplot(0, 1, np.log(mag), normalize=True, title_str='Log Magnitude')
    vis.add_image_subplot(0, 2, pha, normalize=True, title_str='Phase')

show_fft('03/image.jpg')
show_fft('03/image_windows.jpg')
show_fft('03/image_bridge.jpg')
show_fft('03/image_branch.jpg')
show_fft('03/image_lanterns.jpg')


In [ ]:
def create_video(filename, fps, shape):
    return cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (shape[1], shape[0]))

def get_magnitude(dft):
    dft_shift = np.fft.fftshift(dft)
    magnitude = np.clip(np.log(1+np.abs(dft_shift)), 0, None)
    magnitude -= np.min(magnitude)
    magnitude /= np.max(magnitude)
    magnitude *= 255
    return np.uint8(magnitude)

# do some animations

filename = '03/fft_animation_zoom.mp4'
output = create_video(filename, 30, (256, 512))
for i in range(0, 120):
    image = np.zeros((256, 256), dtype=np.uint8)
    s = np.int32(np.sin(i/120 * np.pi * 2) * 32)+1
    cv2.rectangle(image, (64+s, 64+s), (256-64-s, 256-64-s), 255, -1)
    magnitude = get_magnitude(fft.fft2(np.float32(image)))
    frame = np.concatenate([image, np.uint8(magnitude)], axis=1)
    output.write(np.tile(frame[:, :, np.newaxis], (1, 1, 3)))
output.release()
show_video(filename)

filename = '03/fft_animation_rotation.mp4'
output = create_video(filename, 30, (256, 512))
for i in range(0, 120):
    image = np.zeros((256, 256), dtype=np.uint8)
    angle = i/120 * 360
    rect = ((128, 128), (128, 16), angle)
    box = cv2.boxPoints(rect)
    box = np.intp(box)
    cv2.drawContours(image,[box],0,255,-1)
    magnitude = get_magnitude(fft.fft2(np.float32(image)))
    frame = np.concatenate([image, np.uint8(magnitude)], axis=1)
    output.write(np.tile(frame[:, :, np.newaxis], (1, 1, 3)))
output.release()
show_video(filename)

filename = '03/fft_animation_translation.mp4'
output = create_video(filename, 30, (256, 512))
for i in range(0, 120):
    image = np.zeros((256, 256), dtype=np.uint8)
    angle = i/120 * 2 * np.pi
    cx = int(np.sin(angle) * 64 + 128)
    cy = int(np.cos(angle) * 64 + 128)
    rect = ((cx, cy), (128, 16), 0)
    box = cv2.boxPoints(rect)
    box = np.intp(box)
    cv2.drawContours(image,[box],0,255,-1)
    magnitude = get_magnitude(fft.fft2(np.float32(image)))
    frame = np.concatenate([image, np.uint8(magnitude)], axis=1)
    output.write(np.tile(frame[:, :, np.newaxis], (1, 1, 3)))
output.release()
show_video(filename)


# Filtering Examples

In [ ]:
# Box filter

image = cv2.imread('03/image.jpg', cv2.IMREAD_GRAYSCALE)
image_dft = fft.fft2(np.float32(image)/255)
image_mag = get_magnitude(image_dft)
filter = np.zeros(image.shape, dtype=np.uint8)
cv2.rectangle(filter, (image.shape[1]//2-16, image.shape[0]//2-16), (image.shape[1]//2+16, image.shape[0]//2+16), 255, -1)
filter_dft = fft.fft2(np.float32(filter))
filter_mag = get_magnitude(filter_dft)

filtered_dft = image_dft * filter_dft
filtered_image = np.real(fft.fftshift(fft.ifft2(filtered_dft)))
filtered_mag = get_magnitude(fft.fft2(np.float32(filtered_image)))

vis = Visualizer(3, 2, cm='gray', figsize=(10, 10))
vis.add_image_subplot(0, 0, filter, title_str='Box filter')
vis.add_image_subplot(0, 1, filter_mag, normalize=True, title_str='FFT (Magnitude) of Filter')
vis.add_image_subplot(1, 0, image, title_str='Original Image')
vis.add_image_subplot(1, 1, image_mag, normalize=True, title_str='FFT (Magnitude) of Image')
vis.add_image_subplot(2, 0, filtered_image, title_str='Filtered Image')
vis.add_image_subplot(2, 1, filtered_mag, normalize=True, title_str='FFT (Magnitude) of Filtered Image')

In [ ]:
# Gaussian filter

filter = cv2.getGaussianKernel(64, 8)
filter = np.outer(filter, filter)
filter /= np.sum(filter)
pad_size = ((image.shape[0]-filter.shape[0])//2, (image.shape[1]-filter.shape[1])//2)
filter = np.pad(filter, ((pad_size[0],pad_size[0]), (pad_size[1],pad_size[1])), 'constant')
filter_dft = fft.fft2(np.float32(filter))
filter_mag = get_magnitude(filter_dft)

filtered_dft = image_dft * filter_dft
filtered_image = np.real(fft.fftshift(fft.ifft2(filtered_dft)))
filtered_mag = get_magnitude(fft.fft2(np.float32(filtered_image)))


vis = Visualizer(3, 2, cm='gray', figsize=(10, 10))
vis.add_image_subplot(0, 0, filter, title_str='Gaussian filter')
vis.add_image_subplot(0, 1, filter_mag, normalize=True, title_str='FFT (Magnitude) of Filter')
vis.add_image_subplot(1, 0, image, title_str='Original Image')
vis.add_image_subplot(1, 1, image_mag, normalize=True, title_str='FFT (Magnitude) of Image')
vis.add_image_subplot(2, 0, filtered_image, title_str='Filtered Image')
vis.add_image_subplot(2, 1, filtered_mag, normalize=True, title_str='FFT (Magnitude) of Filtered Image')

In [ ]:
# Low-pass filter

mask = np.zeros(image.shape, dtype=np.uint8)
cv2.circle(mask, (image.shape[1]//2, image.shape[0]//2), 64, 255, -1)

filtered_dft = fft.fftshift(image_dft)
filtered_dft[mask == 0] = 0
filtered_image = np.real(fft.ifft2(fft.ifftshift(filtered_dft)))
filtered_mag = get_magnitude(fft.fft2(np.float32(filtered_image)))

vis = Visualizer(2, 2, cm='gray', figsize=(10, 7))
vis.add_image_subplot(0, 0, image, title_str='Original Image')
vis.add_image_subplot(0, 1, image_mag, normalize=True, title_str='FFT (Magnitude) of Image')
vis.add_image_subplot(1, 0, filtered_image, title_str='Filtered Image')
vis.add_image_subplot(1, 1, filtered_mag, normalize=True, title_str='FFT (Magnitude) of Filtered Image')


In [ ]:
# High-pass filter

filtered_dft = fft.fftshift(image_dft)
filtered_dft[mask == 255] = 0
filtered_image = np.real(fft.ifft2(fft.ifftshift(filtered_dft)))
filtered_mag = get_magnitude(fft.fft2(np.float32(filtered_image)))

vis = Visualizer(2, 2, cm='gray', figsize=(10, 7))
vis.add_image_subplot(0, 0, image, title_str='Original Image')
vis.add_image_subplot(0, 1, image_mag, normalize=True, title_str='FFT (Magnitude) of Image')
vis.add_image_subplot(1, 0, filtered_image, title_str='Filtered Image')
vis.add_image_subplot(1, 1, filtered_mag, normalize=True, title_str='FFT (Magnitude) of Filtered Image')